In [1]:
import pandas as pd
from transformers import pipeline

print("Library berhasil diimpor!")

Library berhasil diimpor!


In [2]:
# Muat data dari file CSV yang sudah kita buat
df = pd.read_csv('scraped_articles.csv')

# Tampilkan 5 baris pertama untuk memastikan data termuat dengan benar
print("Data berhasil dimuat:")
df.head()

Data berhasil dimuat:


,title,url,content
0,Janice Tjen Juara Ganda Putri Guangzhou Open 2025,https://www.cnnindonesia.com/olahraga/20251026...,"Petenis tunggal putri Indonesia,Janice Tjen, b..."
1,"Kebijakan Pengelolaan Sumur Minyak Rakyat, Dor...",https://www.cnnindonesia.com/ekonomi/202510251...,Dalam satu tahun kepemimpinan Presiden Prabowo...
2,Smart TV 65 Inch Banting Harga Jadi Rp8 Jutaan...,https://www.cnnindonesia.com/ekonomi/202510241...,"Transmart Full Day Salekembali lagi hari ini, ..."
3,Butet ke Sultan HB X: Kita Panen Keracunan MBG...,https://www.cnnindonesia.com/nasional/20251026...,"Seniman sekaligus budayawan,Butet Kertaredjasa..."
4,Raisa dan Hamish soal Perceraian: Keputusan in...,https://www.cnnindonesia.com/hiburan/202510261...,PasanganHamish DauddanRaisamenyebut pengajuan ...


In [3]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline

model_name = "cahya/t5-base-indonesian-summarization-cased"

try:
    print(f"Mulai memuat komponen untuk model: {model_name}...")

    # Langkah 1: Muat Tokenizer secara manual
    # Ini akan membaca file spiece.model yang sudah Anda unduh
    print("Memuat tokenizer...")
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    print("Tokenizer berhasil dimuat.")

    # Langkah 2: Muat Model secara manual
    # Ini akan memuat file pytorch_model.bin yang sudah Anda unduh
    print("Memuat model (langkah ini mungkin butuh waktu)...")
    model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
    print("Model berhasil dimuat.")
    
    # Langkah 3: Buat pipeline dari komponen yang SUDAH jadi
    # Tidak ada lagi konversi, kita langsung berikan apa yang dibutuhkan
    print("Menginisialisasi pipeline dengan komponen yang sudah ada...")
    summarizer = pipeline("summarization", model=model, tokenizer=tokenizer)
    
    print("\nModel dan Pipeline berhasil diinisialisasi!")

    # --- Contoh Penggunaan ---
    # Mengambil konten artikel pertama dari DataFrame Anda
    text_artikel = df['content'][0] 

    # Hasilkan ringkasan
    print("\nMembuat ringkasan untuk artikel pertama...")
    hasil_ringkasan = summarizer(text_artikel, max_length=100, min_length=30, do_sample=False)

    print("\n--- Hasil Ringkasan ---")
    print(hasil_ringkasan[0]['summary_text'])

except Exception as e:
    print(f"\nTerjadi kesalahan: {e}")

Mulai memuat komponen untuk model: cahya/t5-base-indonesian-summarization-cased...
Memuat tokenizer...


You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


Tokenizer berhasil dimuat.
Memuat model (langkah ini mungkin butuh waktu)...


Device set to use cpu


Model berhasil dimuat.
Menginisialisasi pipeline dengan komponen yang sudah ada...

Model dan Pipeline berhasil diinisialisasi!

Membuat ringkasan untuk artikel pertama...


Both `max_new_tokens` (=256) and `max_length`(=100) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Exception ignored in: <function tqdm.__del__ at 0x00000274B8762F20>
Traceback (most recent call last):
  File "C:\Users\Besto\IndonewsDigest-AutomatedNewsSummarizer\venv\Lib\site-packages\tqdm\std.py", line 1148, in __del__
    self.close()
  File "C:\Users\Besto\IndonewsDigest-AutomatedNewsSummarizer\venv\Lib\site-packages\tqdm\notebook.py", line 279, in close
    self.disp(bar_style='danger', check_delay=False)
AttributeError: 'tqdm' object has no attribute 'disp'



--- Hasil Ringkasan ---
Petenis tunggal putri Indonesia,Janice Tjen, berhasil merebut gelar ganda putri di Guangzhou Open 2025 setelah menang bersama pasangannya, Katarzyna Piter, pada babak final di Nansha International Tennis Center, China, Minggu ( 26/10).


In [4]:
print("Mulai meringkas semua artikel dengan parameter yang disempurnakan...")

summaries = []

for index, row in df.iterrows():
    print(f"Meringkas artikel #{index + 1}: {row['title'][:50]}...")
    
    article_text = row['content']
    
    try:
        # --- PARAMETER YANG DISEMPURNAKAN ---
        summary_result = summarizer(
            article_text, 
            max_length=80,        # Batasi panjang total ringkasan agar lebih padat
            min_length=25,        # Tetapkan panjang minimal agar tidak terlalu pendek
            num_beams=4,          # "Berpikir" dengan 4 kemungkinan cabang (memperbaiki kualitas)
            repetition_penalty=2.0, # Mencegah pengulangan kata
            do_sample=False       # Tetap False untuk hasil yang deterministik
        )
        summaries.append(summary_result[0]['summary_text'])
    except Exception as e:
        print(f"  -> Gagal meringkas artikel #{index + 1}. Error: {e}")
        summaries.append("Gagal diringkas.")

print("\n--- Proses Meringkas Selesai! ---")

# Tambahkan kolom baru (atau timpa yang lama jika Anda menjalankan ulang sel)
df['summary_improved'] = summaries

print("Menampilkan hasil dengan kolom 'summary_improved' baru:")
df.head()

# --- LANGKAH TERAKHIR: Simpan Hasil Anda ---
df.to_csv('articles_with_summaries.csv', index=False)
print("\nFile 'articles_with_summaries.csv' berhasil disimpan!")

Mulai meringkas semua artikel dengan parameter yang disempurnakan...
Meringkas artikel #1: Janice Tjen Juara Ganda Putri Guangzhou Open 2025...


Both `max_new_tokens` (=256) and `max_length`(=80) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Meringkas artikel #2: Kebijakan Pengelolaan Sumur Minyak Rakyat, Dorong ...


Both `max_new_tokens` (=256) and `max_length`(=80) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=256) and `max_length`(=80) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Meringkas artikel #3: Smart TV 65 Inch Banting Harga Jadi Rp8 Jutaan di ...
Meringkas artikel #4: Butet ke Sultan HB X: Kita Panen Keracunan MBG dan...


Both `max_new_tokens` (=256) and `max_length`(=80) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Meringkas artikel #5: Raisa dan Hamish soal Perceraian: Keputusan ini Ka...


Both `max_new_tokens` (=256) and `max_length`(=80) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)



--- Proses Meringkas Selesai! ---
Menampilkan hasil dengan kolom 'summary_improved' baru:

File 'articles_with_summaries.csv' berhasil disimpan!
